In [ ]:
# default_exp distance

# Distance
> Distance calculation

Wiki on [k-d-tree](https://en.wikipedia.org/wiki/K-d_tree)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import numpy as np
import torch
import plotly.graph_objects as go
import plotly.express as px
import timeit
import typing
import itertools
import pandas as pd

## Direct pairwise distances

Example positions and distances

In [ ]:
positions = torch.tensor([[0,0],
                          [1,1],
                          [2,2]], dtype=torch.float)
correct_distances = torch.tensor([2., 8., 2.], dtype=torch.float)

In [ ]:
#export
class PairwiseDistance:
    def __call__(self, x:torch.Tensor, method:str='torch'):
        return getattr(self, f'{method}_pairwise_distance')(x)

In [ ]:
pdist = PairwiseDistance()

Using the Gram matrix to compute pairwise distances: [stackexchange](https://math.stackexchange.com/questions/2240429/pairwise-distance-matrix)

In [ ]:
#export
def pairwise_dist_gram(x:torch.Tensor, y:torch.Tensor,
                       flat:bool=True) -> torch.Tensor:
    
    nx = x.size()[0]
    ny = y.size()[0]
    
    x2 = torch.mm(x, x.t())
    y2 = torch.mm(y, y.t())
    xy = torch.mm(x, y.t())
    
    x2 = x2.diag().unsqueeze(0).expand(ny,-1)
    y2 = y2.diag().unsqueeze(0).expand(nx,-1)
    d = x2.t() + y2 - 2*xy
    if flat:
        ix = torch.triu_indices(nx,ny,offset=1) #list(zip(*list(itertools.combinations(range(len(positions)), r=2))))
        return d[ix[0],ix[1]]
    return d

In [ ]:
%%time
pairwise_dist_gram(positions, positions)

In [ ]:
%%timeit
pairwise_dist_gram(positions, positions)

In [ ]:
#export
def stackoverflow_pairwise_distance(self, x:torch.Tensor):
    return pairwise_dist_gram(x, x)

PairwiseDistance.stackoverflow_pairwise_distance = stackoverflow_pairwise_distance

In [ ]:
pdist(positions, method='stackoverflow')

In [ ]:
%timeit pdist(positions, method='stackoverflow')

In [ ]:
assert torch.isclose(correct_distances, 
              pdist(positions, method='stackoverflow')).all()

Using pre-existing implementation in pytorch: `torch.nn.PairwiseDistance`

In [ ]:
#export
def torch_pairwise_distance(self, x:torch.Tensor):
    nx = ny = x.size()[0]
    pdist = torch.nn.PairwiseDistance(p=2, keepdim=True)
    ix = torch.triu_indices(nx,ny,offset=1)
    return pdist(x[ix[0],:], x[ix[1],:]).squeeze(1) ** 2

PairwiseDistance.torch_pairwise_distance = torch_pairwise_distance

In [ ]:
pdist(positions, method='torch')

In [ ]:
%timeit pdist(positions, method='torch')

In [ ]:
assert torch.isclose(correct_distances, 
              pdist(positions, method='torch')).all()

**Testing the scaling behavior**

In [ ]:
#export
def measure_execution_time(fun:typing.Callable, args, kwargs, 
                           repetitions:int=3, number:int=21):
    return (timeit.Timer(lambda: fun(*args, **kwargs))
            .repeat(repeat=repetitions, number=number))


def get_time_stats(fun:typing.Callable, n:int, d:int, 
                   methods:typing.List[str]=['torch', 'stackoverflow'],
                   reps:int=7, number:int=21):
    
    timing_stats = []
    for method, _n, _d in itertools.product(methods, n, d):
        positions = torch.randn(_n, _d)
        timing_stats.append({
            'n':_n,
            'd':_d,
            'method': method,
            'ts': measure_execution_time(fun, args=(positions,), 
                                         kwargs=dict(method=method),
                                         repetitions=reps,
                                         number=number)
        })
    return pd.DataFrame(timing_stats)

In [ ]:
# number of positions
n = [10, 100, 1000]
# number of dimensions
d = [2, 3, 10]

In [ ]:
%%time
timing_stats = get_time_stats(pdist, n, d)

In [ ]:
timing_stats.head()

In [ ]:
timing_stats_e = timing_stats.explode('ts')

In [ ]:
n_filter = 1000
d_filter = 3

In [ ]:
mask_n = timing_stats['n'] == n_filter
mask_d = timing_stats['d'] == d_filter

In [ ]:
px.scatter(timing_stats_e.loc[mask_d], x='n', y='ts', color='method',
           title=f'Performance @ {d_filter} dims')

In [ ]:
px.scatter(timing_stats_e.loc[mask_n], x='d', y='ts', color='method',
           title=f'Performance @ {n_filter} samples')

TODO:
- mapping between flat and square form to answer nearest neighbor queries

In [ ]:
i = 0
j = 1

num_pos = len(positions)

In [ ]:
class DistMapper:
    
    def __init__(self,num_pos:int):
        self.ix_map = {ix: i for i, ix in enumerate(itertools.combinations(range(num_pos), r=2))}
        self.ix_map_inv = {i: ix for ix, i in self.ix_map.items()}
        self.ix_fun = lambda i,j: int(num_pos*(num_pos-1)*.5  - (num_pos - i)*(num_pos - i - 1)*.5 + j - 1)
    
    def __call__(self,i:int, direction:str, j:int=None):
        return getattr(self,direction)(i,j)
    
    def brute_force_square2flat(self,i:int,j:int):
        if i == j: raise ValueError
        if i > j: i, j = j, i
        return self.ix_map[i,j]
    
    def analytical_square2flat(self,i:int,j:int):
        if i == j: raise ValueError
        if i > j: i, j = j, i
        return self.ix_fun(i,j)
    
    def brute_force_flat2square(self,i:int,j:int=None):
        return self.ix_map_inv[i]

In [ ]:
dmap = DistMapper(num_pos)

In [ ]:
dmap(i,'brute_force_square2flat', j)

In [ ]:
%timeit dmap(i,'brute_force_square2flat', j)

In [ ]:
dmap(i,'analytical_square2flat', j)

In [ ]:
%timeit dmap(i,'analytical_square2flat', j)

In [ ]:
%timeit dmap(i,'brute_force_flat2square')

In [ ]:
dmap(i,'brute_force_flat2square')

In [ ]:
pdist(positions, method='stackoverflow')

stackexchange [solution](https://math.stackexchange.com/questions/2134011/conversion-of-upper-triangle-linear-index-from-index-on-symmetrical-array)

In [ ]:
i = 1
j = 2

num_pos = 9 #len(positions)
print(num_pos)
ix_fun = lambda i,j: int(num_pos*(num_pos-1)*.5  - (num_pos - i)*(num_pos - i - 1)*.5 + j - 1)

In [ ]:
for target, (i,j) in enumerate(itertools.combinations(range(num_pos+1), r=2)):
    print(f'i {i}, j {j}: true {target}, calc {ix_fun(i,j)}')